# Can we use remote sensing to measure benefits of green infrastructure projects in dense urban environments? 


### Outline: 
     Background
        - Benefits of "green infrastructure"
        - Why does this matter?
        - Analysis goals
     Data and analysis 
        - Sentinel-2 geospatial data
        - Location
        - NDMI 
        - Cloud masking
     Results 
     Data Sources



## Background

Stormwater spreading grounds are large-scale green infrastructure projects that help redirect and recapture stormwater runoff from impermeable hardscapes. 

At first glance, these features may appear to have little value in the typical urban environment. They often defined by open areas with little vegetation or defining terrain features, and typically look like a patch of undeveloped of land in an otherwise well-developed urban area. 

### Benefits of "green infrastructure"

So why are these types of projects incorporated so frequently into many large cities? Despite their appearance, these features play a critical role in balancing the interactions between the natural and built environment, including by:  
- Recharging local groundwater resources
- Flattening spikes in peak flow volumes after heavy precipitation
- Reducing pollutant discharge into local water systems 


### Why does this matter?

Sustainable urban development requires looking ahead at the long-term impacts that developed areas have on the natural environment. 

The benefits of green infrastructure projects are becoming increasingly important as cities and urban hubs plan how to mitigate the adverse effects of climate change, particularly on water availability and changing preciptation norms, over the next century.  

### Goals

This analysis will help measure and validate the role of stormwater spreading grounds in the Los Angeles region through measurement of moisture level and  local precipitation data. 


In [1]:
import os

from PIL import Image

!pip install utm
import utm

# Run functions nb
%run functions.ipynb

from glob import glob
from datetime import datetime
import nbconvert

import earthpy.spatial as es
import earthpy.plot as ep
import earthpy.mask as em
import earthpy as et

import scipy.stats
from scipy import stats

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from matplotlib.dates import MonthLocator, DateFormatter
from matplotlib.axes._axes import _log as matplotlib_axes_logger

from itertools import groupby
from netCDF4 import Dataset
    
import geopandas as gpd
import rasterio as rio
import xarray as xr
import rioxarray as rxr
from shapely.geometry import mapping
import numpy.ma as ma
from rasterio.plot import plotting_extent
import rasterstats as rs
from shapely.geometry import box

%matplotlib inline

# Download data and set working directory
os.chdir(os.path.join(et.io.HOME, 
                      'earth-analytics', 
                      'urban-stormwater-capture'))

NameError: name 'test_rxr_8' is not defined

NameError: name 'test_rxr_8' is not defined

#### Below: Pacoima Spreading Grounds in Los Angeles, CA after heavy rainfall


In [ ]:
image = Image.open(os.path.join("graphics",
                                "Pacoima-SG1.jpg"))

image.thumbnail((600, 600))
image.save('image_thumbnail.jpg')

image

## Data and Analysis 


### Sentinel-2 geospatial data
This analysis will use Sentinel-2 HLS data, which provides multiple reflective light bands at a 30-meter spatial resolution. 


### Measuring NDMI 
To view the impact of green infrastucture projects on stormwater capture, we will use the Near-Infared (NIR) and Short-Wave Infared (SWIR) Sentinel-2 bands to calculate the Normalized Difference Moisture Index (NDMI) of both project areas and the surrounding region. 

*NDMI = (Band 5 (NIR) – Band 6 (SWIR)) / (Band 5 (NIR) + Band 6 (SWIR))*

NDMI is primarily used to determine vegetation water content. Ideally, these project areas will show greater effectiveness in capturing and retaining stormwater runoff through longer efficacy of saturated soils and vegetation growth post-precipitation. 

#### How exactly does NDMI correlate with moisture levels? 

This table below shows how the NDMI measurement relates to different levels of soil saturation, vegetation, and/or water stress for a given area:

| NDMI | Interpretation |
|   :---   |   :---  |
|-1 – -0.8 |Bare soil |
|-0.8 – -0.6 | Almost absent canopy cover |
|-0.6 – -0.4 | Very low canopy cover |
|-0.4 – -0.2 |Low canopy cover, dry or very low canopy cover, wet |
|-0.2 – 0 |Mid-low canopy cover, high water stress or low canopy cover, low water stress |
|0 – 0.2 |Average canopy cover, high water stress or mid-low canopy cover, low water stress |
|0.2 – 0.4 |Mid-high canopy cover, high water stress or average canopy cover, low water stress |
|0.4 – 0.6 |High canopy cover, no water stress |
|0.6 – 0.8 |Very high canopy cover, no water stress |
|0.8 – 1 |Total canopy cover, no water stress/waterlogging |

In [ ]:
# set up paths to data

# LA spreading grounds shapefiles
shape_spread_path = os.path.join("data",
                                 "la-spreading-grounds",
                                 "Spreading_Grounds_Feature_Layer.shp")                                 
shape_spread_data = gpd.read_file(shape_spread_path)

#LA catchment areas shapefiles
shape_catchment_path = os.path.join("data",
                                    "la-catchment-areas",
                                    "Los_Angeles_County_Catchment_Areas.shp")                                 
shape_catchment_data = gpd.read_file(shape_catchment_path)

#LA area region crop 
la_crop_path = os.path.join("data",
                       "Sentinel-2-Shapefile-Index",
                       "sentinel_2_index_shapefile.shp")

la_crop_data = gpd.read_file(la_crop_path).loc[
                                            gpd.read_file(la_crop_path)["Name"]=="11SLT"]

#City boundaries shapefiles

city_boundaries_path = os.path.join("data",
                                    "la-city-boundaries",
                                    "City_Boundaries.shp")   

city_boundaries_data = gpd.read_file(city_boundaries_path)

city_boundaries_data.rename(columns = {'CITY_NAME':'NAME'}, inplace = True)


#LA streams and rivers shapefiles 
la_streams_rivers_path = os.path.join("data",
                                     "la-streams-rivers",
                                     "geo_export_339f28ec-e123-431b-a506-4d994dfd8153.shp")
la_streams_rivers_data = gpd.read_file(la_streams_rivers_path)


#Sentinel-2 data paths

sent_2019_2021_path = os.path.join("data", 
                                  "sent-2019-2021")
sent_landsat_hls_path = os.path.join("data",
                                    "sent-landsat-hls")

#Select path that will be used primarily for analysis
sent_path = os.path.join(sent_2019_2021_path)


# precipitation data paths

ges_precip_path = os.path.join("data",
                                "ges-precip-data")

#Paths to access NIR (Band 8) and SWIR (Band 11) 
band_8_paths = glob(os.path.join(sent_path,"*B08*.tif")) 
band_11_paths = glob(os.path.join(sent_path,"*B11*.tif"))
qa_layer_paths = glob(os.path.join(sent_path,"*Fmask*.tif"))

landsat_band_5 = glob(os.path.join(sent_landsat_hls_path,"*B05*.tif"))
landsat_band_6 = glob(os.path.join(sent_landsat_hls_path,"*B06*.tif"))
landsat_qa_layer = glob(os.path.join(sent_landsat_hls_path,"*Fmask*.tif"))

      

ndmi_band_paths = band_8_paths + band_11_paths
all_band_paths = band_8_paths + band_11_paths + qa_layer_paths

sent_landsat_band_paths = landsat_band_5 + landsat_band_6 + landsat_qa_layer

In [ ]:

### Follow Sentinel-2 HLS documentation to create masking value references ## 

# Git: https://git.earthdata.nasa.gov/projects/LPDUR/repos/hls-tutorial/browse/HLS_Tutorial.ipynb#processhls

def cloud_mask (fmask):
    
    """
    Create bit dictionary and map Sentinel-2 HLS quality assurance (QA) layers from bits
    

    """

    # bitword dictionaries 

    bitword_order = (1, 1, 1, 1, 1, 1, 2)  # set the number of bits per bitword
    num_bitwords = len(bitword_order)      # Define the number of bitwords based on your input above
    total_bits = sum(bitword_order)        # Should be 8, 16, or 32 depending on datatype

    # Loop to create cloud mask lookup from Git documentation 

    qVals = list(np.unique(fmask))  # Create a list of unique values that need to be converted to binary and decoded

    all_bits = list()
    summary_bits=list()

    goodQuality = []
    for v in qVals:
        all_bits = []
        bits = total_bits
        i = 0

        # Convert to binary based on the values and # of bits defined above:
        bit_val = format(v, 'b').zfill(bits)
#         print('\n' + str(v) + ' = ' + str(bit_val))
        all_bits.append(str(v) + ' = ' + str(bit_val))

        # Go through & split out the values for each bit word based on input above:
        for b in bitword_order:
            prev_bit = bits
            bits = bits - b
            i = i + 1
            if i == 1:
                bitword = bit_val[bits:]
#                 print(' Bit Word ' + str(i) + ': ' + str(bitword)) ## use to print visual tally of bitmap 
                all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword)) 
            elif i == num_bitwords:
                bitword = bit_val[:prev_bit]
#                 print(' Bit Word ' + str(i) + ': ' + str(bitword)) ## use to print visual tally of bitmap 
                all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword))
            else:
                bitword = bit_val[bits:prev_bit]
#                 print(' Bit Word ' + str(i) + ': ' + str(bitword)) ## use to print visual tally of bitmap 
                all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword))

#       1 = Cirrus, 2 = Cloud, 3 = Cloud adjacent, 4 = cloud shadow, 5 = snow/ice, 6 = water, 7 = aerosol
        if int(all_bits[1].split(': ')[-1]) < 1 and \
            int(all_bits[2].split(': ')[-1]) < 1 and \
            int(all_bits[3].split(': ')[-1]) < 1 and \
            int(all_bits[4].split(': ')[-1]) < 2 and \
            int(all_bits[5].split(': ')[-1]) < 1 and \
            int(all_bits[6].split(': ')[-1]) < 2 and \
                int(all_bits[7].split(': ')[-1]) < 12:
        
            goodQuality.append(v)
            
    # test values that do not need to be masked 
    return goodQuality

In [ ]:
def open_clean_bands(band_path,
                     crop_bound,
                     valid_range,
                    pixel_qa_path):
    
    """
    Open, clean, and crop a file into an xarray DataArray.

    Parameters
    -----------
    band_path:string
        A path to the array to be opened
    crop_bound:geopandas GeoDataFrame
        A geopandas dataframe to be used to crop the raster data using rioxarray clip().
    valid_range:tuple (optional)
         A tuple of min and max range of values for the data. Default = None

    Returns
    -----------

    band : xarray DataArray
        End result for a cropped xarray dataarray
    """

#     crop_bound_box = [box(*crop_bound.total_bounds)]

    try:
        band = rxr.open_rasterio(band_path,
                                 masked=True).rio.clip(
                                                        crop_bound.geometry,
                                                        crs=crop_bound.crs,
                                                        all_touched=True,
                                                        from_disk=True).squeeze()
    
        mask = ((band < valid_range[0]) | (band > valid_range[1]))
        band = band.where(~xr.where(mask, True, False))

        # open and mask cloud layer
        cl_mask = (rxr.open_rasterio(pixel_qa_path, masked=True)
                        .rio.clip(
                            crop_bound.geometry,
                            from_disk=True).squeeze())

        # apply cloud mask
        band = band.where(cl_mask.isin(goodQuality)).squeeze()
        
    except:
        raise ValueError(
            "Oops - I couldn't clip your data. This may be due to a crs error.")

    return band

In [ ]:


# test bands for a single site 

#band 8
test_tif_NIR = os.path.join("data",
                          "sent-landsat-hls",
                        "HLS.L30.T11SLT.2017032T182822.v2.0.B05.tif")

#band 11
test_tif_SWIR = os.path.join("data", 
                           "sent-landsat-hls",
                        "HLS.L30.T11SLT.2017032T182822.v2.0.B06.tif")

#create rasters for bands 8 and 11
test_rxr_NIR = rxr.open_rasterio(test_tif_NIR,masked=True).squeeze()

test_rxr_SWIR = rxr.open_rasterio(test_tif_SWIR,masked=True).squeeze()

In [ ]:
#reproject shape files to crs 
la_reproj_crop = la_crop_data.to_crs(test_rxr_SWIR.rio.crs)

#spreading ground reprojections for Eaton Spreading Grounds

spreading_shape_reproj = shape_spread_data.to_crs(test_rxr_SWIR.rio.crs)

eaton_shape = spreading_shape_reproj.loc[
                                        spreading_shape_reproj['NAME'] == "EATON S.G."]


#Reprojections for other potential shapefiles 

city_boundaries_reproj = city_boundaries_data.to_crs(test_rxr_SWIR.rio.crs)

pasadena_shape = city_boundaries_reproj.loc[
                                        city_boundaries_reproj['NAME'] == "Pasadena"]


#create plot extents for site (LA) and project area (Eaton SG)

la_plot_extent = plotting_extent(test_rxr_SWIR, 
                                   test_rxr_SWIR.rio.transform())


### Where is our project area? 

This analysis focuses primarly on the Eaton Stormwater Spreading Grounds. This stormwater spreading ground site is located in Pasadena - part of the Los Angeles metropolitan area. 

In [ ]:
import folium 
la_map = folium.Map(location=[34.0522, -118.2437])
folium.GeoJson(data=eaton_shape).add_to(la_map)

la_map

In [ ]:
eaton_shape

In [ ]:
pasadena_shape

In [ ]:
# Create path for the pixel_qa_layer for subdirectory scene
scene_pixel_qa_path = os.path.join(sent_landsat_hls_path,
                                  "HLS.L30.T11SLT.2017032T182822.v2.0.Fmask.tif")

sent_qa = rxr.open_rasterio(scene_pixel_qa_path).squeeze()

#define valid values to be exluded from mask
goodQuality = cloud_mask(fmask = sent_qa)

### Cloud Masking

Cloud masks necessary to remove data values impacted by clouds or other atmospheric interference from satellite imagery. Sentinel-2 HLS provides an "Fmask" quality assurance (QA) band along with each data file, along with documentation on how to properly utilize different masking features.  

Git: https://git.earthdata.nasa.gov/projects/LPDUR/repos/hls-tutorial/browse/HLS_Tutorial.ipynb#processhls

#### An example of masking "bad" data pixels

In [ ]:

# #test ndmi
test_ndmi_xr = (test_rxr_NIR-test_rxr_SWIR) / (test_rxr_NIR+test_rxr_SWIR)

masked_test_NIR = open_clean_bands(test_tif_NIR, 
                                 crop_bound=la_reproj_crop,
                                 valid_range=(0,10000),
                                 pixel_qa_path=scene_pixel_qa_path)

# test single band raster plot and ndmi plots  
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,15))

ep.plot_bands(test_rxr_NIR,
              ax=ax1,
              extent=la_plot_extent,
              cbar="terrain_r",
             title = "Single band plot")

eaton_shape.plot(ax=ax1,
                       color='none',
                       linewidth = 3,
                       edgecolor = 'purple')

ep.plot_bands(masked_test_NIR,
              ax=ax2,
              extent=la_plot_extent,
              cbar="terrain_r",
             title = "Cloud mask plot")

eaton_shape.plot(ax=ax2,
                color='none',
                linewidth = 3,
                edgecolor = 'purple')


plt.tight_layout()
plt.show()

## Results 

First, we create a loop to provide list of the Sentinel-2 NIR and SWIR bands, indexed by date. 

In [ ]:
# Create loop to process a sample Sentinel-2 view 
band_list = []

# for tifs in all_band_paths:
for tifs in sent_landsat_band_paths:


    filename = os.path.normpath(tifs).split(os.sep)[-1]
    name_components = filename.split(".")
    site = name_components[2]
    band = name_components[6]
    old_date = name_components[3][0:7]

    datetimeobject = datetime.strptime(old_date,'%Y%j')
    date = datetimeobject.strftime('%m-%d-%Y')
    
    output = [date, band, tifs]
        
    band_list.append(output)
    
band_df = pd.DataFrame(band_list,
                   columns = ["date","band","array"])

band_df['date'] = pd.to_datetime(band_df.date)

indexed_df = band_df.set_index("date").sort_values(by="date")

print(indexed_df.duplicated().sum())
indexed_df

Then, we pivot the table to group the selected bands by the indexed date.  

In [ ]:
band_df.pivot_table(index = "date", columns = "band", values = "array", aggfunc='first')

Finally, we write a loop to run through all of the NIR and SWIR arrays listed in the dataframe, open and mask any cloud interference, calculate NDMI and other key summary statistics, then place into two dataframes.

In [ ]:
#Calculate NDMI through normalized difference of band 8 and 11

shape_list = [eaton_shape, pasadena_shape]


band_pivot = band_df.pivot_table(index = "date", columns = "band", values = "array", aggfunc='first')


mean_list = []
ndmi_arrays = []
for i, row in band_pivot.iterrows():
    #define bands and perform normalized diff for ndmi
    band_NIR_path = row["B05"]    
    band_SWIR_path = row["B06"]
    mask_path = row["Fmask"]
    
    #loop for both sites
    for shape in shape_list: 

        area_name = shape["NAME"].item()

    #create fmask layer from qa layer path
        band_qa_layer = rxr.open_rasterio(mask_path).squeeze()
        goodQuality = cloud_mask(fmask = band_qa_layer)
        
    # mask NIR and SWIR bands
        masked_band_NIR = open_clean_bands(band_NIR_path,
                                         crop_bound=shape, 
                                         valid_range=(0,10000),
                                         pixel_qa_path=mask_path)

        masked_band_SWIR = open_clean_bands(band_SWIR_path,
                                          crop_bound=shape, 
                                          valid_range=(0,10000),
                                          pixel_qa_path=mask_path)
        
    #calculate normalized difference for ndmi
        ndmi = (masked_band_NIR-masked_band_SWIR) / (masked_band_NIR+masked_band_SWIR)
        
        #create summary values for box plot 
        ndmi_box = ndmi.to_dataframe(name="box")
        stats_list = ndmi_box["box"].dropna().values

        try:
            min_ndmi = np.min(stats_list)
            first_q = np.percentile(stats_list,25)
            second_q = np.percentile(stats_list,50)
            third_q = np.percentile(stats_list,75)
            max_ndmi = np.max(stats_list)
        except ValueError:
            pass
               
    
    #count number of unmasked pixels to validate data from scene
        clean_pixels = ndmi.count().item()
        total_pixels = ndmi.size
        
        mean_ndmi = ndmi.mean().item()
        
    #Create outputs for mean ndmi list (and ndmi arrays just in case) 
        output_1 = [i, mean_ndmi,area_name, clean_pixels]
        mean_list.append(output_1)

        output_2 = [i,ndmi, masked_band_NIR, masked_band_SWIR,
                    area_name,clean_pixels,
                    min_ndmi, first_q, second_q,third_q,max_ndmi]
        
        ndmi_arrays.append(output_2)

        
#Create dataframes for mean NDMI and NDMI arrays
mean_df = pd.DataFrame(mean_list,
                           columns = ["date","mean","area","clean_pixels"])

mean_df["total_pixels"] = max(mean_df["clean_pixels"])

mean_df

Many of the scene data are heavily obscured by clouds - some of which slip through the cloud mask function. 

To catch these potential outliers, we calculate the proportion of masked pixels to total pixels and "drop" the scenes that were heavily obscured by clouds. This will limit the amount of outliers in our final data results. 

In [ ]:
#set total pixel counts for both areas based off thier individual maxes

max_eaton_pixel = max(mean_df.loc[mean_df['area'] == "EATON S.G."]["clean_pixels"])
max_pasadena_pixel = max(mean_df.loc[mean_df['area'] == "Pasadena"]["clean_pixels"])

mean_df['total_pixels'] = np.where((mean_df.area =="EATON S.G."),max_eaton_pixel,max_pasadena_pixel)


# calculate proportion of good pixels 
mean_df["clean_proportion"] = mean_df["clean_pixels"] / mean_df["total_pixels"]

mean_ndmi_df = mean_df.set_index("date")

ndmi_df = pd.DataFrame(ndmi_arrays,
                           columns = ["date","ndmi","NIR_band","SWIR_band","area","clean_pixels",
                                      "ndmi_min","1Q_ndmi","2Q_ndmi","3Q_ndmi","max_ndmi"])

# add proportion for ndmi dataframe as well
ndmi_df['clean_proportion'] = mean_df["clean_proportion"]


In [ ]:
# drop data results where they were heavily impacted by cloud cover 

filter_value = .75

mean_ndmi_df["clean_mean"] = mean_ndmi_df["mean"].where(mean_ndmi_df["clean_proportion"] >= filter_value, axis = None)
ndmi_df = ndmi_df.drop(ndmi_df[ndmi_df['clean_proportion'] <= filter_value].index)


# reset index

pasadena_ndmi_df = ndmi_df.loc[ndmi_df['area'] == "Pasadena"].reset_index()
eaton_ndmi_df = ndmi_df.loc[ndmi_df['area'] == "EATON S.G."].reset_index()




#### Viewing the NIR, SWIR, and NDMI Arrays for both Project and Surrounding Areas

With the core loop complete, let's first put some eyes on the resulting arrays to make sure everything visually checks out before starting our final analysis.  

In [ ]:
last_five = [61,62,63,64]
first_two = [0,1]

for num in first_two: 

    fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(15, 5))

    ndmi_df['NIR_band'][num].plot.imshow(ax=ax1, 
                                        cmap = "Greys")
    eaton_shape.plot(ax=ax1,
                     color='none',
                     linewidth = 1,
                     edgecolor = 'purple',
                     vmin=-1,
                     vmax=-1)

    ndmi_df['SWIR_band'][num].plot.imshow(ax=ax2, 
                                        cmap = "Greys")    
    eaton_shape.plot(ax=ax2,
                     color='none',
                     linewidth = 1,
                     edgecolor = 'purple',
                     vmin=-1,
                     vmax=-1)
    
    ndmi_df['ndmi'][num].plot.imshow(ax=ax3, 
                                    cmap = "terrain_r",
                                    vmin=-1,
                                    vmax=1)
    
    eaton_shape.plot(ax=ax3,
                     color='none',
                     linewidth = 1,
                     edgecolor = 'purple',
                     vmin=-1,
                     vmax=-1)
    
    plt.suptitle(ndmi_df['date'][num])

    plt.tight_layout()


#### Visual comparison of mean NDMI for the Project Area vs. Surrounding Area 

In [ ]:
first_five = [0,1,2,3,4]

for num in first_five:

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8, 4))

    eaton_ndmi_df['ndmi'][num].plot.imshow(ax=ax1, 
                                            cmap = "terrain_r",
                                            vmin=-1,
                                            vmax=1)
    eaton_shape.plot(ax=ax1,
                     color='none',
                     linewidth = 1,
                     edgecolor = 'red',
                     vmin=-1,
                     vmax=-1)

    pasadena_ndmi_df['ndmi'][num].plot.imshow(ax=ax2, 
                                            cmap = "terrain_r",
                                             vmin=-1,
                                             vmax=1)
    pasadena_shape.plot(ax=ax2,
                    color='none',
                    linewidth = 1,
                    edgecolor = 'grey',
                    vmin=-1,
                    vmax=-1)

    eaton_shape.plot(ax=ax2,
                     color='none',
                     linewidth = 1,
                     edgecolor = 'grey',
                     vmin=-1,
                     vmax=-1)

    plt.suptitle(pasadena_ndmi_df['date'][num])
    plt.tight_layout()
    plt.show()

### Adding in Precipitation Data

Lastly, we'll want to include IMERG precipication data for these same dates and project/site areas to see how the NDMI measurements correspond with precipitation volume. 

In [ ]:
nc4_test = os.path.join(ges_precip_path,
                        "3B-DAY-L.MS.MRG.3IMERG.20220513-S000000-E235959.V06.nc4.SUB.nc4")

all_precip_paths = glob(os.path.join(ges_precip_path,
                                    "*"))

appended_precip_list = []

# precip_name_components
for file in all_precip_paths:

    precip_filename = os.path.normpath(file).split(os.sep)[-1]
    precip_name_components = precip_filename.split(".")
    
    precip_date = precip_name_components[4][0:8]

    precip_datetime = datetime.strptime(precip_date, '%Y%m%d').strftime('%m/%d/%Y')
    
    metadata = xr.open_dataset(file)
    metadata_df = metadata.to_dataframe()
    metadata_df.reset_index(inplace=True)

        
    appended_precip_list.append(metadata_df)
    
precip_df = pd.concat(appended_precip_list)


precip_df


While our subsetted data from GES roughly around the LA region, we'll want to make sure we select the areas that are as close as possible to the centeroid of the shapefiles for both project and site areas. 

In [ ]:

# determine aoi bounds on utm scale     
aoi_lat_y = [float(eaton_shape.total_bounds[1]), float(eaton_shape.total_bounds[3])]
aoi_lon_x = [float(eaton_shape.total_bounds[0]), float(eaton_shape.total_bounds[2])]

aoi_centroid = [sum(aoi_lon_x)/2, sum(aoi_lat_y)/2]
aoi_centroid_utm = utm.to_latlon(aoi_centroid[0],aoi_centroid[1], 11, 'N')

#select precip coords near aoi centroid
precip_df_aoi = precip_df[(precip_df['lat'].between(34.14,34.15)) &
                                   (precip_df['lon'].between(-118.1,-118)) & 
                                    (precip_df['bnds'] == (0))]
                                                                 
                                    
precip_df_aoi.set_index("time",inplace=True)

# precip_df_aoi = precip_df_aoi.join(mean_ndmi_df["mean"])

precip_df_aoi

### Time-series comparison of mean NDMI values 

Finally - let's put the data together in full and analyze the results! 

In [ ]:
fig, ax1 = plt.subplots(figsize=(10,4)) #consider a qqplot or probplot?

ax1.scatter(mean_ndmi_df[mean_ndmi_df["area"] == "EATON S.G."].index, 
        mean_ndmi_df[mean_ndmi_df["area"] == "EATON S.G."]["clean_mean"],
        label="Eaton S.G.",
        color="green",
        s=10)

ax1.scatter(mean_ndmi_df[mean_ndmi_df["area"] == "Pasadena"].index, 
        mean_ndmi_df[mean_ndmi_df["area"] == "Pasadena"]["clean_mean"],
        label="Pasadena (city)",
        color="black",
        s=10)

# add precip data on same graph but opposite x axis
ax2 = ax1.twinx()
ax2.plot(precip_df_aoi.index, 
        precip_df_aoi["precipitationCal"],
        label="Precipitation (mm)",
        color="lightblue")

ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left',
          prop={'size': 11})

ax2.legend(bbox_to_anchor=(1.05, .8), loc='upper left',
          prop={'size': 11})

ax1.set(xlabel = "Date",
       ylabel = "Mean NDMI")

ax2.set_ylabel('Precipitation \n (mm)')


plt.setp(ax1.get_xticklabels(), rotation = 30)

ax2.xaxis.tick_top()
ax2.xaxis.set_label_position('top') 

ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax1.xaxis.set_major_formatter(DateFormatter("%b %Y"))



plt.title("Comparison of Mean NDMI \n Eaton Spreading Grounds vs Surrounding Area (Pasadena) ")

In [ ]:
#set index back to date for both ndmi arrays

pasadena_ndmi_df.set_index("date", inplace = True)
eaton_ndmi_df.set_index("date",inplace = True)

### NDMI Arrays 

However, looking at the mean NDMI only can be decieving. For one, taking the mean value of a large sample size also ends up "normalizing" some of the high and low variance results - particularly in the case of the city of Pasadena where the sample size is so large! 

To observe the full impact of these precipitation events, let's look at all NDMI values from each scene's arrays:

In [ ]:

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(18,6)) #consider a qqplot or probplot?

columns = ["ndmi_min","1Q_ndmi","2Q_ndmi","3Q_ndmi","max_ndmi"]
count = [0,1,2,3,4]

for num in count:     
    if columns[num] == "ndmi_min" or columns[num] == "max_ndmi":
        linewidth = .5
        alpha = .5
        marker = None

    elif columns[num] == "1Q_ndmi" or columns[num] == "3Q_ndmi":
        linewidth = 2
        alpha = .5
        marker = None
    else:
        linewidth = 4
        alpha = 1
        marker = None


    ax1.plot(eaton_ndmi_df.index, 
             eaton_ndmi_df[columns[num]],
            label=columns[num],
            linestyle="-",
            alpha = alpha,
            color = "green",
            linewidth = linewidth,
            marker = marker)
    
    ax2.plot(pasadena_ndmi_df.index, 
            pasadena_ndmi_df[columns[num]],
            label=columns[num],
            linestyle="-",
            alpha = alpha,
            color = "black",
            linewidth = linewidth,
            marker = marker)


# set labels, legends, and axis
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left',
          prop={'size': 11})
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left',
          prop={'size': 11})


ax1.set(xlabel = "Date",
       ylabel = "NDMI",
       title = "NDMI Range - Eaton Stormwater Spreading Grounds")
ax2.set(xlabel = "Date",
       ylabel = "NDMI",
       title = "NDMI Range - Surrounding Area (Pasadena)")


plt.setp(ax1.get_xticklabels(), rotation = 30)
plt.setp(ax2.get_xticklabels(), rotation = 30)


ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=4))
ax1.xaxis.set_major_formatter(DateFormatter("%b %Y"))

ax2.xaxis.set_major_locator(mdates.MonthLocator(interval=4))
ax2.xaxis.set_major_formatter(DateFormatter("%b %Y"))

plt.tight_layout()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,5)) #consider a qqplot or probplot?

columns = ["ndmi_min","1Q_ndmi","2Q_ndmi","3Q_ndmi","max_ndmi"]
count = [0,1,2,3,4]

for num in count:     
    if columns[num] == "ndmi_min" or columns[num] == "max_ndmi":
        linewidth = 1
        alpha = .75

    elif columns[num] == "1Q_ndmi" or columns[num] == "3Q_ndmi":
        linewidth = 2
        alpha = .75
        marker = None
    else:
        linewidth = 4
        alpha = 1
        marker = None
        
    ax1.plot(eaton_ndmi_df.index, 
             eaton_ndmi_df[columns[num]],
            label=columns[num],
            linestyle="-",
            alpha = alpha,
            color = "green",
            linewidth = linewidth,
            marker = marker)
    
    ax2.plot(pasadena_ndmi_df.index, 
             pasadena_ndmi_df[columns[num]],
            label=columns[num],
            linestyle="-",
            alpha = alpha,
            color = "black",
            linewidth = linewidth,
            marker = marker)

    
    ax3 = ax1.twinx()
    ax4 = ax2.twinx()
    ax3.plot(precip_df_aoi.index, 
        precip_df_aoi["precipitationCal"],
        label="Precip. (mm)",
        color="lightblue",
        alpha = .5)
    
    ax4.plot(precip_df_aoi.index, 
        precip_df_aoi["precipitationCal"],
        label="Precip. (mm)",
        color="lightblue",
        alpha=.5)


# set labels, legends, and axis
ax1.legend(bbox_to_anchor=(1.05, .5), loc='lower left',
          prop={'size': 12})
ax2.legend(bbox_to_anchor=(1.05, .5), loc='lower left',
          prop={'size': 12})

ax3.legend(bbox_to_anchor=(1.05, .4), loc='lower left',
          prop={'size': 12})

ax4.legend(bbox_to_anchor=(1.05, .4), loc='lower left',
          prop={'size': 12})


ax1.set(xlabel = "Date",
       ylabel = "NDMI",
       title = "Eaton S.G. with Precipitation (mm)")

ax2.set(xlabel = "Date",
       ylabel = "NDMI",
       title = "Pasadena with Precipitation (mm)")

plt.setp(ax1.get_xticklabels(), rotation = 30)
plt.setp(ax2.get_xticklabels(), rotation = 30)


ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=4))
ax1.xaxis.set_major_formatter(DateFormatter("%b %Y"))

ax2.xaxis.set_major_locator(mdates.MonthLocator(interval=4))
ax2.xaxis.set_major_formatter(DateFormatter("%b %Y"))

plt.tight_layout()

### Findings

The comparison of mean NDMI between the Eaton Stormwater Spreading Grounds and surrounding Pasadena area uncovers a few interesting data insights: 
1. **The urban area surrounding the stormwater spreading grounds has a higher mean NDMI throughout the year** 
    - This is likely due to the fact that the city of Pasadena (like most urban & residential areas) has lots of irrigated vegetation - residential lawns, commercial landscaping, tree-lined streets, and greenspaces like parks all make an impact! 
    - Conversely, the Eaton Spreading Grounds is mostly comprised of bare earth and sparse vegetation. It makes sense for these mean NDMI values to be lower - especially in an arid region like Los Angeles!  
    
2. **The stormwater spreading grounds appear to have greater variability - particularly in response to precipitation events**
    - The rainy seasons in the winters of 2017, 2019, and 2020 all show steep spikes in mean NDMI for the Eaton Spreading Grounds
    
    
3. **Pasadena's variance in NDMI appear to be more correlated with season, while Eaton Spreading Ground's variance seems to be more directly correlated to precipitation response** 
    - This is likely a result of the Eaton Spreading Grounds receiving an deluge of stormwater from surrounding urban hardscape during these storm events - which is then recaptured into local groundwater resources!  


4. **Overall, NDMI may be imperfect for adequately measuring stormwater capture for urban green infrastucture projects**
    - Sentinel-2 data was chosen for it's high spatial resolution (30m) in order to measure projects within an urban environment.
    - However, NDMI is best suited for measuing vegetation-related moisture rather than soil moisture levels.
    - A more specific soil moisture measurement like SMAP would provide better direct correlation to the ability of stormwater spreading grounds to capture and reclaim groundwater resources, but it has too low of a spatial resolution to gather effective data for all but the largest types of stormwater spreading grounds
    - NDMI can provide some insight, but catches results from many other moisture-related variables (particularly from vegetation) that may skew the intended analysis 
    
  


In [ ]:
!jupyter nbconvert --to html  --no-input ea-capstone-workbook-2022.ipynb

!mv ea-capstone-workbook-2022.ipynb urban-stormwater-capture-blog.html

##  Data Sources 

- Sentinel-2 HLS Data: https://search.earthdata.nasa.gov/search
- GES Precipitation Data: https://search.earthdata.nasa.gov/search
- City of Los Angeles Open Data: https://data.lacity.org/
- City of Los Angeles GeoHub: https://geohub.lacity.org/ 